# Industrial Transformers

## Libraries

Based off of [Vanilla Transformers](https://colab.research.google.com/drive/1VAsHQLrCSNb4V_c-mCFdIYfQBXXIQYz0#scrollTo=_QWiUFmzTkXL)


In [2]:
!nvidia-smi

Sat Apr 22 21:29:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8    N/A /  N/A |      6MiB /  4040MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import math
from numbers import Number
import re
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
from dataclasses import dataclass, field

import jax
import jax.numpy as jnp
from jax import random

import optax
from jax import lax
import flax
from flax import linen as nn

# from flax import optim
from flax import jax_utils
from flax.training import train_state, checkpoints, common_utils

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from functools import partial


# Data Preprocessing

- Drop all completely empty columns.
- Make local_* variables strings.
- Scale numeric variables.
- Drop id and date variables.
- Separate strings using.


In [3]:
weather = pl.read_parquet("../data/weather_clean.parquet")
print(weather.shape)
weather.sample(5)


(635664, 25)


x,y,station_name,climate_identifier,province_code,local_year,local_month,local_day,local_hour,temp,temp_flag,dew_point_temp,dew_point_temp_flag,humidex,precip_amount,precip_amount_flag,relative_humidity,relative_humidity_flag,station_pressure,station_pressure_flag,wind_chill,wind_direction,wind_direction_flag,wind_speed,wind_speed_flag
f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,str,f64,str,f64,str,f64,f64,str,f64,str
-111.213333,56.651111,"""FORT MCMURRAY …","""3062696""","""AB""","""2013""","""5""","""18""","""12""",22.6,"""missing""",-1.9,"""missing""",null,null,"""missing""",19.0,"""missing""",96.88,"""missing""",null,9.0,"""missing""",13.0,"""missing"""
-114.004722,49.521667,"""PINCHER CREEK …","""3035208""","""AB""","""2016""","""2""","""16""","""9""",6.1,"""missing""",-2.7,"""missing""",null,0.0,"""missing""",53.0,"""missing""",87.17,"""missing""",null,25.0,"""missing""",52.0,"""missing"""
-112.7675,49.695,"""LETHBRIDGE CDA…","""3033890""","""AB""","""2016""","""5""","""25""","""3""",4.4,"""missing""",3.9,"""missing""",null,0.0,"""missing""",96.0,"""missing""",90.66,"""missing""",null,19.0,"""missing""",8.0,"""missing"""
-111.213333,56.651111,"""FORT MCMURRAY …","""3062696""","""AB""","""2016""","""7""","""30""","""15""",22.0,"""missing""",12.4,"""missing""",null,0.0,"""missing""",54.0,"""missing""",96.07,"""missing""",null,4.0,"""missing""",19.0,"""missing"""
-113.605836,53.306667,"""EDMONTON INTER…","""3012206""","""AB""","""2011""","""7""","""20""","""22""",8.8,"""missing""",8.8,"""missing""",null,null,"""missing""",100.0,"""missing""",null,"""M""",null,null,"""M""",null,"""M"""


In [4]:
energy = pl.read_parquet("../data/energy_clean.parquet")
print(energy.shape)
energy.sample(5)


(111743, 10)


local_year,local_month,local_day,local_hour,total_energy_no_imports,total_imports,total_exports,actual_pool_price,actual_ail,day_ahead_pool_price
str,str,str,str,f64,f64,f64,f64,i64,f64
"""2012""","""12""","""6""","""22""",7090.006864,250.0,0.0,38.16,9545,36.91
"""2011""","""2""","""9""","""4""",6435.332551,245.0,100.0,21.33,8356,23.54
"""2017""","""10""","""15""","""17""",7361.620082,0.0,150.0,18.29,9250,18.29
"""2022""","""3""","""18""","""23""",6595.433734,300.0,0.0,43.71,9528,41.88
"""2013""","""10""","""9""","""10""",7204.043731,45.0,0.0,857.29,8897,889.35


Scale numeric variables and remove spaces from strings.


In [5]:
def scale_numeric(df):
    for col in df.columns:
        if df[col].dtype == pl.Float64 or df[col].dtype == pl.Int64:
            df = df.with_columns(
                ((pl.col(col) - pl.col(col).mean()) / pl.col(col).std()).alias(col)
            )  # .select(pl.col(["dew_point_temp", "NewCOL"]))
    return df


weather = scale_numeric(weather)
energy = scale_numeric(energy)


In [6]:
def make_lower_remove_special_chars(df):
    df = df.with_columns(
        pl.col(pl.Utf8).str.to_lowercase().str.replace_all("[^a-zA-Z0-9]", " ")
    )
    return df


weather = make_lower_remove_special_chars(weather)
energy = make_lower_remove_special_chars(energy)


In [7]:
def get_unique_utf8_values(df):
    arr = np.array([])
    for col in df.select(pl.col(pl.Utf8)).columns:
        arr = np.append(arr, df[col].unique().to_numpy())

    return np.unique(arr)


weather_val_tokens = get_unique_utf8_values(weather)
energy_val_tokens = get_unique_utf8_values(energy)
energy_val_tokens


array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4',
       '5', '6', '7', '8', '9'], dtype=object)

In [8]:
def get_col_tokens(df):
    tokens = []
    for col_name in df.columns:
        sub_strs = re.split(r"[^a-zA-Z0-9]", col_name)
        tokens.extend(sub_strs)
    return np.unique(np.array(tokens))


weather_col_tokens = get_col_tokens(weather)
energy_col_tokens = get_col_tokens(energy)
energy_col_tokens


array(['actual', 'ahead', 'ail', 'day', 'energy', 'exports', 'hour',
       'imports', 'local', 'month', 'no', 'pool', 'price', 'total',
       'year'], dtype='<U7')

In [9]:
special_tokens = np.array(
    [
        "missing",
        "<batch-start>",
        "<batch-end>",
        "<pad>",
        "<unk>",
        ":",
        ",",
        "<row-start>",
        "<row-end>",
    ]
)
tokens = np.unique(
    np.concatenate(
        (
            weather_val_tokens,
            energy_val_tokens,
            weather_col_tokens,
            energy_col_tokens,
            special_tokens,
        )
    )
)
tokens


array([',', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30',
       '3012206', '3026knq', '3031094', '3033890', '3035208', '3062696',
       '31', '4', '5', '6', '7', '8', '9', ':', '<batch-end>',
       '<batch-start>', '<pad>', '<row-end>', '<row-start>', '<unk>',
       'ab', 'actual', 'ahead', 'ail', 'amount', 'calgary int l cs',
       'chill', 'climate', 'code', 'day', 'dew', 'direction',
       'edmonton international cs', 'energy', 'exports', 'flag',
       'fort mcmurray cs', 'hour', 'humidex', 'humidity', 'identifier',
       'imports', 'lethbridge cda', 'local', 'm', 'missing', 'month',
       'name', 'no', 'pincher creek climate', 'point', 'pool', 'precip',
       'pressure', 'price', 'province', 'relative', 'speed', 'station',
       'sundre a', 'temp', '

In [10]:
@dataclass
class StringNumeric:
    value: Union[str, float]
    # all_tokens: np.array
    is_numeric: bool = field(default=None, repr=True)
    embedding_idx: int = field(default=None, repr=True)

    def __post_init__(self):
        if isinstance(self.value, str):
            self.is_numeric = False
        else:
            self.is_numeric = True
            self.embedding_idx = 0

    def gen_embed_idx(self, tokens: np.array):
        if not self.is_numeric:
            try:
                self.embedding_idx = np.where(tokens == self.value)[0][0] + 1
            except IndexError:
                self.embedding_idx = np.where(tokens == "<unk>")[0][0] + 1


x = StringNumeric(value="climate")
# xx = StringNumeric(value="climate", tokens=tokens)
print(x)
y = StringNumeric(value=1.0)
print(y)
z = StringNumeric(value="SomeRandomString")
print(z)
x.gen_embed_idx(tokens)
print(x)
# print(StringNumeric(value=1.0, all_tokens=tokens))


StringNumeric(value='climate', is_numeric=False, embedding_idx=None)
StringNumeric(value=1.0, is_numeric=True, embedding_idx=0)
StringNumeric(value='SomeRandomString', is_numeric=False, embedding_idx=None)
StringNumeric(value='climate', is_numeric=False, embedding_idx=67)


In [11]:
tokens


array([',', '0', '1', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '2', '20', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30',
       '3012206', '3026knq', '3031094', '3033890', '3035208', '3062696',
       '31', '4', '5', '6', '7', '8', '9', ':', '<batch-end>',
       '<batch-start>', '<pad>', '<row-end>', '<row-start>', '<unk>',
       'ab', 'actual', 'ahead', 'ail', 'amount', 'calgary int l cs',
       'chill', 'climate', 'code', 'day', 'dew', 'direction',
       'edmonton international cs', 'energy', 'exports', 'flag',
       'fort mcmurray cs', 'hour', 'humidex', 'humidity', 'identifier',
       'imports', 'lethbridge cda', 'local', 'm', 'missing', 'month',
       'name', 'no', 'pincher creek climate', 'point', 'pool', 'precip',
       'pressure', 'price', 'province', 'relative', 'speed', 'station',
       'sundre a', 'temp', '

In [12]:
class TabularDataset(Dataset):
    # def __init__(self, df: pl.DataFrame, vocab_dict: Dict, m_dim: int) -> Dataset:
    def __init__(
        self,
        df: pl.DataFrame,
        vocab,
        shuffle_cols=False,
        n_rows=None,
        max_seq_length=512,
    ) -> Dataset:
        self.df = df
        self.vocab = vocab
        self.shuffle_cols = shuffle_cols
        self.n_rows = n_rows
        self.max_seq_length = max_seq_length
        # self.vocab_dict = vocab_dict
        # self.embedding = nn.Embedding(len(self.string_vocab), m_dim)
        # Numeric Scale

        # self.col_vocab = self.df.columns

    def __len__(self):
        """Returns the number of sequences in the dataset."""
        length = self.df.shape[0] // self.n_rows
        return length

    def __getitem__(self, idx):
        """Returns a tuple of (input, target) at the given index."""
        batch = self.batch(idx)
        start = StringNumeric("<batch-start>")
        start.gen_embed_idx(self.vocab)
        end = StringNumeric("<batch-end>")
        end.gen_embed_idx(self.vocab)
        batch = self.padder(batch)
        batch = [start] + batch + [end]
        return batch

    def batch(self, idx):
        """Returns a batch from splitter from the starting index to the start
        index + n_rows"""
        batch = []
        for i in range(idx, idx + self.n_rows):
            row = self.df[i]
            row = self.splitter(row)
            batch.extend(row)

        return batch

    def padder(self, batch: List[StringNumeric]):
        diff = self.max_seq_length - len(batch)  # -2 for start and end
        if diff > 0:
            pad = StringNumeric("<pad>")
            pad.gen_embed_idx(self.vocab)
            batch.extend([pad] * diff)
        elif diff < 0:
            batch = batch[: self.max_seq_length - 1]
            # add warning
            new_end = StringNumeric("<batch-end>")
            new_end.gen_embed_idx(self.vocab)
            batch.append(new_end)
            print("Batch too long, truncating")
            Warning("Batch too long, truncating")
        return batch

    def splitter(self, row: pl.DataFrame) -> List[Union[str, float, None]]:
        vals = ["<row-start>"]
        cols = row.columns
        if self.shuffle_cols:
            np.random.shuffle(cols)

        for col in cols:
            value = row[col][0]
            col = col.split("_")
            vals.extend(col)
            vals.append(":")
            if isinstance(value, Number):
                vals.append(value)
            elif value is None:
                vals.append("missing")
                # Nones are only for numeric columns, others are "None"
            elif isinstance(value, str):
                vals.extend(value.split(" "))
            else:
                raise ValueError("Unknown type")
            vals.append(",")
        vals.append("<row-end>")

        vals = [StringNumeric(value=val) for val in vals]
        for val in vals:
            val.gen_embed_idx(self.vocab)

        return vals


weather_ds = TabularDataset(weather, tokens, shuffle_cols=False, n_rows=2)
energy_ds = TabularDataset(energy, tokens, shuffle_cols=False, n_rows=2)
print(weather_ds[0][:10])


[StringNumeric(value='<batch-start>', is_numeric=False, embedding_idx=55), StringNumeric(value='<row-start>', is_numeric=False, embedding_idx=58), StringNumeric(value='x', is_numeric=False, embedding_idx=103), StringNumeric(value=':', is_numeric=False, embedding_idx=53), StringNumeric(value=-0.551099305737714, is_numeric=True, embedding_idx=0), StringNumeric(value=',', is_numeric=False, embedding_idx=1), StringNumeric(value='y', is_numeric=False, embedding_idx=104), StringNumeric(value=':', is_numeric=False, embedding_idx=53), StringNumeric(value=-0.37817406811183396, is_numeric=True, embedding_idx=0), StringNumeric(value=',', is_numeric=False, embedding_idx=1)]


In [13]:
@dataclass
class Config:
    embed_dim: int = 256
    n_heads: int = 8
    kvq_dim: int = embed_dim // n_heads
    ff_dim: int = 512
    p_drop: float = 0.2
    encoder_vocab_size: int = weather_ds.vocab.shape[0]
    decoder_vocab_size: int = energy_ds.vocab.shape[0]


config = Config()


In [14]:
# jnp.arange(10) * jnp.arange(10).T#[:, None]
# jnp.einsum("i,j->ij", jnp.arange(10), jnp.arange(10))
jnp.arange(10)[jnp.newaxis, :].ndim  # [:, None]


2

In [15]:
class PositionalEncoding(nn.Module):
    """Adds positional encoding to the input."""

    config: Config

    @nn.compact
    def __call__(self, x: jnp.array) -> jnp.array:
        assert x.ndim == 3, "Input must have rank 3"
        config = self.config

        batch_size, seq_len, _ = x.shape
        pe = jnp.zeros(batch_size, seq_len, config.embed_dim)

        position = jnp.arrange(seq_len)[jnp.newaxis, :]
        div_term = jnp.exp(
            jnp.arange(0, config.embed_dim, 2) * (-jnp.log(10000.0) / config.embed_dim)
        )
        radians = jnp.einsum(
            "ij,kl->jl", position, div_term
        )  # just a matrix multiplication
        pe = pe.at[:, :, 0::2].set(jnp.sin(radians))
        pe = pe.at[:, :, 1::2].set(jnp.cos(radians))
        return (x + pe).astype(jnp.float32)


In [16]:
class MultiHeadAttention(nn.Module):
    """Multi Headed Dot Product Attention"""

    config: Config

    @nn.compact
    def __call__(
        self,
        q: jnp.array,
        k: jnp.array,
        v: jnp.array,
        mask: jnp.array = None,
        dropout: float = 0.0,
    ) -> jnp.array:
        config = self.config

        assert q.ndim == k.ndim == v.ndim == 3, "Input must have rank 3"
        assert q.shape[0] == k.shape[0] == v.shape[0], "Batch size must be equal"
        assert (
            q.shape[2] == k.shape[2] == v.shape[2]
        ), "Embedding dimension must be equal"

        batch_size, seq_len, embed_dim = q.shape
        assert (
            embed_dim % config.num_heads == 0
        ), "Embedding dimension must be divisible by number of heads"

        q = nn.Dense(config.embed_dim, name="DenseQ")(q)
        k = nn.Dense(config.embed_dim, name="DenseK")(k)
        v = nn.Dense(config.embed_dim, name="DenseV")(v)

        q = q.reshape(-1, seq_len, config.n_heads, config.kvq_dim)
        k = k.reshape(-1, seq_len, config.n_heads, config.kvq_dim)
        v = v.reshape(-1, seq_len, config.n_heads, config.kvq_dim)

        attention = jnp.einsum("...qhd,...khd->...hqk", q, k) / jnp.sqrt(config.kvq_dim)

        if mask is not None:
            attention = jnp.where(mask, attention, -jnp.inf)

        attention = nn.softmax(attention, axis=-1)
        values = jnp.einsum("...hqk,...khd->...qhd", attention, v)
        values = values.reshape(-1, seq_len, embed_dim)
        out = nn.Dense(embed_dim)(values)

        return out, attention


In [17]:
class FeedForward(nn.Module):
    """Feed Forward Neural Network"""

    config: Config

    @nn.compact
    def __call__(self, x: jnp.array, deterministic: bool) -> jnp.array:
        config = self.config
        x = nn.Dense(config.ff_dim * 4, name="FFDense1")(x)
        x = nn.relu(x)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        x = nn.Dense(config.ff_dim, name="FFDense2")(x)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)

        return x


In [18]:
class TransformerEncoderLayer(nn.Module):
    """Transformer Encoder Layer"""

    config: Config

    @nn.compact
    def __call__(self, x: jnp.array, mask: jnp.array, deterministic: bool) -> jnp.array:
        config = self.config
        res = x
        x, attention = MultiHeadAttention(config)(x, x, x, mask)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        x = nn.LayerNorm(name="LayerNorm1")(x + res)
        res = x
        x = FeedForward(config, name="FeedForward")(x, deterministic=deterministic)
        x = nn.LayerNorm(name="LayerNorm2")(x + res)

        return x, attention


In [19]:
class TransformerDecoderLayer(nn.Module):
    config: Config
    """Transformer Decoder Layer"""

    @nn.compact
    def __call__(
        self,
        x: jnp.array,
        memory: jnp.array,
        decoder_mask: jnp.array,
        encoder_decoder_mask: jnp.array,
        deterministic: bool,
    ) -> Tuple[jnp.array, jnp.array, jnp.array]:
        config = self.config
        res = x
        x, attention = MultiHeadAttention(config)(x, x, x, decoder_mask)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        x = nn.LayerNorm(name="LayerNorm1")(x + res)
        res = x
        x, attention = MultiHeadAttention(config)(
            x, memory, memory, encoder_decoder_mask
        )
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        x = nn.LayerNorm(name="LayerNorm2")(x + res)
        return x, attention


In [20]:
x = jnp.array([1, 1, 3, 3, 5, 6, 7, 8, 9, 10])

emb = nn.Embed(num_embeddings=10, features=5)
emb_variables = emb.init(random.PRNGKey(0), x)
emb_output = emb.apply(emb_variables, x)
emb_output


Array([[-0.440659  ,  0.23434746,  0.44432253,  0.35004553, -1.0721016 ],
       [-0.440659  ,  0.23434746,  0.44432253,  0.35004553, -1.0721016 ],
       [-0.5751345 , -0.31515005, -0.7112598 , -0.22740227,  0.52951103],
       [-0.5751345 , -0.31515005, -0.7112598 , -0.22740227,  0.52951103],
       [-0.1656743 ,  0.0334887 ,  0.7145505 , -0.69273764, -0.7260953 ],
       [-0.12573542, -0.35354394,  0.10587429,  0.05312492,  0.4437487 ],
       [-0.04921824, -0.10327773, -0.13198015, -0.4364049 , -0.38484678],
       [ 0.31554   ,  0.09454814,  0.4258944 , -0.27779824,  0.07495691],
       [ 0.71339667, -0.04502202, -0.35073858,  0.59638804,  0.05645175],
       [        nan,         nan,         nan,         nan,         nan]],      dtype=float32)

In [21]:
emb_output


Array([[-0.440659  ,  0.23434746,  0.44432253,  0.35004553, -1.0721016 ],
       [-0.440659  ,  0.23434746,  0.44432253,  0.35004553, -1.0721016 ],
       [-0.5751345 , -0.31515005, -0.7112598 , -0.22740227,  0.52951103],
       [-0.5751345 , -0.31515005, -0.7112598 , -0.22740227,  0.52951103],
       [-0.1656743 ,  0.0334887 ,  0.7145505 , -0.69273764, -0.7260953 ],
       [-0.12573542, -0.35354394,  0.10587429,  0.05312492,  0.4437487 ],
       [-0.04921824, -0.10327773, -0.13198015, -0.4364049 , -0.38484678],
       [ 0.31554   ,  0.09454814,  0.4258944 , -0.27779824,  0.07495691],
       [ 0.71339667, -0.04502202, -0.35073858,  0.59638804,  0.05645175],
       [        nan,         nan,         nan,         nan,         nan]],      dtype=float32)

In [22]:
y = jnp.arange(10)
print(y)
y = y.at[2].set(100)
print(y)


[0 1 2 3 4 5 6 7 8 9]
[  0   1 100   3   4   5   6   7   8   9]


In [23]:
class FloatEmbedding(nn.Module):
    # config: Config
    """Embedding lookup for token ids."""

    @nn.compact
    def __call__(self, x: list[StringNumeric]) -> jnp.array:
        config = Config()
        embed = nn.Embed(
            config.vocab_size,
            config.embed_dim,
            embedding_init=jax.nn.initializers.normal(stddev=config.embed_dim**-0.5),
            name="FloatEmbedding",
        )
        embeddings = jnp.zeros((len(x), config.embed_dim))
        # return embeddings
        for i, sn in enumerate(x):
            if sn.is_numeric:
                arr = jnp.zeros(config.embed_dim)
                arr = arr.at[0].set(sn.value)
                embeddings = embeddings.at[i].set(arr)
            else:
                embeddings = embeddings.at[i].set(embed(sn.embedding_idx))

        return embeddings


In [24]:
class TransformerEncoder(nn.Module):
    config: Config
    """Transformer Encoder"""

    @nn.compact
    def __call__(
        self,
        x: jnp.array,
        mask: jnp.array,
        deterministic: bool,
        return_attention: bool = False,
    ) -> Union[jnp.array, Tuple[jnp.array, List[jnp.array]]]:
        config = self.config
        x = FloatEmbedding(config)(x)
        x = PositionalEncoding(config)(x)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        attention_list = []
        for i in range(config.n_layers):
            x, attention = TransformerEncoderLayer(config, name=f"EncoderLayer_{i}")(
                x, mask, deterministic
            )
            attention_list.append(attention)

        x = nn.LayerNorm(name="LayerNorm")(x)
        if return_attention:
            return x, attention_list

        return x


In [25]:
class TransformerDecoder(nn.Module):
    config: Config
    """Transformer Decoder"""

    @nn.compact
    def __call__(
        self,
        x: jnp.array,
        memory: jnp.array,
        decoder_mask: jnp.array,
        encoder_decoder_mask: jnp.array,
        deterministic: bool,
        return_attention: bool = False,
    ) -> Union[jnp.array, Tuple[jnp.array, List[jnp.array]]]:
        config = self.config
        x = FloatEmbedding(config)(x)
        x = PositionalEncoding(config)(x)
        x = nn.Dropout(config.p_drop)(x, deterministic=deterministic)
        self_attention_list, src_attention_list = [], []
        for i in range(config.n_layers):
            x, self_attention, src_attention = TransformerDecoderLayer(config, name=f"DecoderLayer_{i}")(
                x,
                memory,
                decoder_mask,
                encoder_decoder_mask,
                deterministic,
            )
            self_attention_list.append(self_attention)
            src_attention_list.append(src_attention)


        x = nn.LayerNorm(name="LayerNorm")(x)
        x = nn.Dense(config.vocab_size, name="Dense")(x)
        if return_attention:
            return x, self_attention_list, src_attention_list

        return x

In [26]:
class Transformer(nn.Module):
    config: Config
    """Transformer"""

    def setup(self):
        self.Encoder = TransformerEncoder(self.config)
        self.Decoder = TransformerDecoder(self.config)



        def encode(self, src: TabularDataset, train: bool = True) -> jnp.array:
            config = self.config

            encoder_mask = nn.make_attention_mask(jnp.ones_like(src), src, dtype="bool")


        def __call__(
                self,
                src: TabularDataset,
                tgt: TabularDataset,
                train: bool = True
        ) -> jnp.array:
            

            memory 

In [27]:
flax.__version__

'0.6.8'

In [28]:
help(nn.make_attention_mask)

Help on function make_attention_mask in module flax.linen.attention:

make_attention_mask(query_input: Any, key_input: Any, pairwise_fn: Callable[..., Any] = <PjitFunction of <function jax.numpy.multiply at 0x7f84cff83ac0>>, extra_batch_dims: int = 0, dtype: Any = <class 'jax.numpy.float32'>)
    Mask-making helper for attention weights.
    
    In case of 1d inputs (i.e., `[batch..., len_q]`, `[batch..., len_kv]`, the
    attention weights will be `[batch..., heads, len_q, len_kv]` and this
    function will produce `[batch..., 1, len_q, len_kv]`.
    
    Args:
      query_input: a batched, flat input of query_length size
      key_input: a batched, flat input of key_length size
      pairwise_fn: broadcasting elementwise comparison function
      extra_batch_dims: number of extra batch dims to add singleton
        axes for, none by default
      dtype: mask return dtype
    
    Returns:
      A `[batch..., 1, len_q, len_kv]` shaped mask for 1d attention.



In [31]:
help(partial)

Help on class partial in module functools:

class partial(builtins.object)
 |  partial(func, *args, **keywords) - new function with partial application
 |  of the given arguments and keywords.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __reduce__(...)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |  
 |  __setstate__(...)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__(...) from builtins.type
 |      See PEP 585
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwa